In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Cargar el archivo CSV
# Asegúrate de que el nombre del archivo coincida con el que subiste.
file_path = "data/prueba modelo cuadrante negocio.xlsx"
df = pd.read_excel(file_path)

# Renombrar columnas para facilitar el manejo (basado en el snippet de datos)
df.columns = [
    'CUADRANTE',
    'MES_N',
    'TOTAL_ROBONe_MES_N',
    'TOTAL_ROBONe_MES_N_MENOS_1',
    'TOTAL_ROBONe_MES_N_MENOS_2'
]

# Mostrar las primeras filas
print("### 1. Datos Originales (Primeras 5 Filas) ###")
print(df.head())
print("-" * 50)

### 1. Datos Originales (Primeras 5 Filas) ###
   CUADRANTE  MES_N  TOTAL_ROBONe_MES_N  TOTAL_ROBONe_MES_N_MENOS_1  \
0          1      1                   2                           1   
1          2      1                   0                           0   
2          3      1                  13                          11   
3          4      1                   2                           3   
4          5      1                   0                           1   

   TOTAL_ROBONe_MES_N_MENOS_2  
0                           0  
1                           0  
2                          12  
3                           6  
4                           0  
--------------------------------------------------


In [6]:
# Reestructurar los datos para el modelo de predicción de 78 salidas
# Vamos a agrupar los datos por MES_N.

# La 'Y' (salida) serán los 78 robos del Mes N, apilados en un array.
Y_data = df.groupby('MES_N')['TOTAL_ROBONe_MES_N'].apply(lambda x: x.values).tolist()
Y = np.array(Y_data)

# Las 'X' (entradas) serán los 78 cuadrantes, el mes N-1 y el mes N-2.
# En este caso, usaremos el CUADRANTE, N-1 y N-2 para predecir N.
X_features = df[['CUADRANTE', 'TOTAL_ROBONe_MES_N_MENOS_1', 'TOTAL_ROBONe_MES_N_MENOS_2']]
X_data = X_features.groupby(df['MES_N']).apply(lambda x: x.values.flatten()).tolist()
X = np.array(X_data)

# No excluir la primera fila: al agregar por MES_N sobre varios años, las columnas N-1/N-2
# contienen las sumas (p.ej. diciembre y noviembre) y por tanto son válidas.
# Solo garantizamos que X e Y tengan la misma cantidad de filas.
if X.shape[0] != Y.shape[0]:
    min_len = min(X.shape[0], Y.shape[0])
    X = X[:min_len]
    Y = Y[:min_len]

# Definir las dimensiones
input_dim = X.shape[1]  # 78 * 3 = 234 (variables de entrada)
output_dim = Y.shape[1] # 78 (variables de salida)

# Dividir para entrenamiento y prueba (usaremos el último mes para probar)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.15, shuffle=False # Usamos shuffle=False para mantener el orden temporal
)

print(f"Dimensión de Entrada (X): {X.shape}")
print(f"Dimensión de Salida (Y): {Y.shape}")
print("-" * 50)

Dimensión de Entrada (X): (12, 234)
Dimensión de Salida (Y): (12, 78)
--------------------------------------------------


In [7]:
# 1. Construir el modelo Keras
model = Sequential([
    # Capa de entrada (tamaño de la capa = Dimensión de entrada * 2, por ejemplo)
    Dense(256, activation='relu', input_shape=(input_dim,)),
    Dropout(0.2), # Regularización para evitar sobreajuste

    # Capa oculta
    Dense(128, activation='relu'),
    Dropout(0.2),

    # Capa de salida (debe tener 78 neuronas, una para cada cuadrante)
    Dense(output_dim, activation='linear') # 'linear' para problemas de regresión
])

# 2. Compilar el modelo
# Usamos 'mean_squared_error' (MSE) como función de pérdida, estándar en regresión.
# Optimizador Adam es eficiente.
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

print("### 2. Estructura del Modelo Keras ###")
model.summary()
print("-" * 50)

# 3. Entrenar el modelo
# Usamos EarlyStopping para detener el entrenamiento cuando el modelo deje de mejorar
# y evitar sobreajuste.
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("### 3. Entrenamiento del Modelo (puede tardar un momento) ###")
history = model.fit(
    X_train, Y_train,
    epochs=100,
    batch_size=4,
    validation_data=(X_test, Y_test),
    callbacks=[early_stop],
    verbose=0 # 0 para no mostrar el progreso en cada época
)
print("Entrenamiento finalizado. Mejores pesos restaurados.")
print("-" * 50)

### 2. Estructura del Modelo Keras ###


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 256)            │        60,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 78)             │        10,062 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 103,118 (402.80 KB)

 Trainable params: 103,118 (402.80 KB)

 Non-trainable params: 0 (0.00 B)

--------------------------------------------------
### 3. Entrenamiento del Modelo (puede tardar un momento) ###
Entrenamiento finalizado. Mejores pesos restaurados.
--------------------------------------------------


In [8]:
# 1. Generar Predicciones en el conjunto de prueba
Y_pred_test = model.predict(X_test)

# 2. Calcular R^2
# Para R^2 en un modelo de múltiples salidas, se compara el valor real Y_test
# (aplanado) con el valor predicho Y_pred_test (aplanado).
r2 = r2_score(Y_test.flatten(), Y_pred_test.flatten())

print(f"### 4. Evaluación del Modelo ###")
print(f"Coeficiente de Determinación (R^2): {r2:.4f}")

if r2 > 0.7:
    print("✅ ¡El modelo tiene una precisión alta!")
elif r2 > 0.5:
    print("⚠️ Precisión moderada. Podría requerir más capas o datos.")
else:
    print("❌ Precisión baja. Se recomienda ajustar hiperparámetros o la estructura del modelo.")

print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
### 4. Evaluación del Modelo ###
Coeficiente de Determinación (R^2): 0.8922
✅ ¡El modelo tiene una precisión alta!
--------------------------------------------------


In [ ]:
# Suponiendo que el Mes 11 es el último mes completo en tus datos.
# El número del mes N-1 es:
mes_n_menos_1 = df['MES_N'].max() 

# 1. Filtrar la data del Mes N-1 (el mes de entrada para la predicción)
df_entrada_mes = df[df['MES_N'] == mes_n_menos_1]

# 2. Seleccionar las columnas de entrada y aplanarlas en una sola fila
X_predict_raw = df_entrada_mes[['CUADRANTE', 'TOTAL_ROBO_MES_N_MENOS_1', 'TOTAL_ROBO_MES_N_MENOS_2']].values

# Aplanar el array 78x3 a un array 1D de 234 elementos y reformarlo
X_predict = X_predict_raw.flatten().reshape(1, -1) 

# Asegurarse de que el tamaño es correcto (debe ser 234)
if X_predict.shape[1] == 234:
    print(f"✅ Entrada de predicción (X_predict) lista con forma: {X_predict.shape}")

    # 3. Realizar la predicción
    Y_pred_mes_12 = model.predict(X_predict)
    
    # 4. Obtener el resultado específico para el Cuadrante 78
    # El cuadrante 78 corresponde a la posición 77 (índice 0-based).
    robo_c78_m12 = Y_pred_mes_12[0][77]
    
    print(f"Robo predicho para el Mes 12, Cuadrante 78: {robo_c78_m12:.2f}")

else:
    print(f"❌ Error: La matriz de entrada no tiene 234 variables. Forma: {X_predict.shape}")

In [11]:
# quiero predecir el numero de robos para todos los meses y para todos los cuadrantes
# usando los datos de los meses anteriores
for mes in range(1, 2): # mes 1
    for cuadrante in range(1, 79):  # Cuadrantes de 1 a 78
        # Filtrar la data del Mes N-1 (el mes de entrada para la predicción)
        df_entrada_mes = df[df['MES_N'] == mes - 1]
        # Seleccionar las columnas de entrada y aplanarlas en una sola fila
        X_predict_raw = df_entrada_mes[['CUADRANTE', 'TOTAL_ROBONe_MES_N_MENOS_1', 'TOTAL_ROBONe_MES_N_MENOS_2']].values
        X_predict = X_predict_raw.flatten().reshape(1, -1)
        # Asegurarse de que el tamaño es correcto (debe ser 234)
        if X_predict.shape[1] == 234:
            # Realizar la predicción
            Y_pred_mes = model.predict(X_predict)
            # Obtener el resultado específico para el cuadrante actual
            robo_cuadrante = Y_pred_mes[0][cuadrante - 1]
            print(f"Robo predicho para el Mes {mes}, Cuadrante {cuadrante}: {robo_cuadrante:.2f}")

In [ ]:
# 1. Des-estructurar las predicciones y re-insertarlas en el DataFrame

# Asumimos que Y_test corresponde a los últimos meses del DataFrame original (df).
# El tamaño de Y_pred_test es el número de meses en la prueba * 78 cuadrantes.

# El índice de inicio en el DataFrame original es donde comienzan los datos de prueba
start_index = len(df) - len(Y_test.flatten())

# Crear la columna de Predicción (inicialmente NaN para los datos de entrenamiento)
df['PREDICCION_MES_N'] = np.nan

# Insertar las predicciones en la sección de prueba del DataFrame
df.loc[start_index:, 'PREDICCION_MES_N'] = Y_pred_test.flatten()

print("### 5. DataFrame con la Columna de Predicción ###")
print(df.tail(80)) # Mostramos los últimos meses (donde hay predicciones)
print("-" * 50)

In [ ]:
# 2. Identificar los 5 Cuadrantes de Mayor Incidencia para el mes a predecir
# La última predicción es la que corresponde al 'Mes N' más alto que predijo
# (que es el último mes de prueba en este caso).

# Obtenemos el último conjunto de 78 predicciones.
ultimas_predicciones = df['PREDICCION_MES_N'].tail(78).to_frame()
ultimas_predicciones['CUADRANTE'] = df['CUADRANTE'].tail(78).values
ultimas_predicciones.columns = ['PREDICCION_ROBOS', 'CUADRANTE']

# Ordenar por predicción descendente
top_cuadrantes = ultimas_predicciones.sort_values(by='PREDICCION_ROBOS', ascending=False).head(5)

print("### 6. TOP 5 Cuadrantes con Mayor Incidencia Delictiva (Mes Predicho) ###")
print(top_cuadrantes)

In [ ]:
print(model.predict(np.array([[13, 45, 37]])))